In [33]:
# Dependencies
import os, cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, preprocessing
from sklearn.model_selection import train_test_split

In [23]:
# Read training and testing images into arrays
realTrain = []
fakeTrain = []
realTest = []
fakeTest = []

for filename in os.listdir("./data/train/REAL"):
    imgPath = os.path.join("./data/train/REAL", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            realTrain.append(img)

for filename in os.listdir("./data/train/FAKE"):
    imgPath = os.path.join("./data/train/FAKE", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            fakeTrain.append(img)

for filename in os.listdir("./data/test/REAL"):
    imgPath = os.path.join("./data/test/REAL", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            realTest.append(img)

for filename in os.listdir("./data/test/FAKE"):
    imgPath = os.path.join("./data/test/FAKE", filename)
    if os.path.isfile(imgPath):
        img = cv2.imread(imgPath)
        if img is not None:
            fakeTest.append(img)

realTrain = np.array(realTrain)
fakeTrain = np.array(fakeTrain)
realTest = np.array(realTest)
fakeTest = np.array(fakeTest)

xTrain = np.concatenate((realTrain, fakeTrain), axis=0)
yTrain = np.concatenate((np.ones(len(realTrain)), np.zeros(len(fakeTrain))))
xTest = np.concatenate((realTest, fakeTest), axis=0)
yTest = np.concatenate((np.ones(len(realTest)), np.zeros(len(fakeTest))))

print(f"""
realTrain legnth: {len(realTrain)}
fakeTrain length: {len(fakeTrain)}
realTest length: {len(realTest)}
fakeTest length: {len(fakeTest)}
xTrain length: {len(xTrain)}
yTrain length: {len(yTrain)}
xTest length: {len(xTest)}
yTest length: {len(yTest)}""")


realTrain legnth: 50000
fakeTrain length: 50000
realTest length: 10000
fakeTest length: 10000
xTrain length: 100000
yTrain length: 100000
xTest length: 20000
yTest length: 20000


In [47]:
# Train and test the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(xTrain, yTrain, epochs=20, validation_split=0.2)

testLoss, testAccuracy = model.evaluate(xTest, yTest)
trainLoss, trainAccuracy = model.evaluate(xTrain, yTrain)
print(f"Test Loss: {testLoss}\nTest Accuracy: {testAccuracy}")
print(f"Train Loss: {trainLoss}\nTrain Accuracy: {trainAccuracy}")

Epoch 1/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7947 - loss: 0.7885 - val_accuracy: 0.8278 - val_loss: 0.3809
Epoch 2/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8877 - loss: 0.2737 - val_accuracy: 0.8624 - val_loss: 0.3299
Epoch 3/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9013 - loss: 0.2443 - val_accuracy: 0.8594 - val_loss: 0.3289
Epoch 4/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9130 - loss: 0.2203 - val_accuracy: 0.9732 - val_loss: 0.0807
Epoch 5/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9179 - loss: 0.2038 - val_accuracy: 0.9103 - val_loss: 0.2243
Epoch 6/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9237 - loss: 0.1920 - val_accuracy: 0.9047 - val_loss: 0.2370
Epoch 7/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9306 - loss: 0.1751 - val_accuracy: 0.9208 - val_loss: 0.1985
Epoch 8/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9348 - loss: 0

In [54]:
# Save and export the model
model.save("img_classifier.keras")